# Анализ вариантов обслуживания автобусов на основе имитационной модели

In [2]:
from examples.load_packages import load_packages

load_packages()

In [3]:
from packages.bas_models.src.bus_generator import BusGenerator
#Вероятность поломок
generator = BusGenerator([0.4, 0.7])
#Создаем автобус с заданной вероятностью поломок
buses = generator.generate(1)
flights = 10 #рейсы
#Вероятности поломок
probs = [
    [0.0,0.05],
    [0.1,0.05],
    [0.2,0.11],
    [0.3,0.11],
    [0.4,0.15],
    [0.5,0.15],
    [0.6,0.19],
    [0.7,0.19],
    [0.8,0.23],
    [0.9,0.23],
    [1.0,0.7]]

In [4]:
from packages.bas_models.src.model import Model

def arrange_statistics(model: Model, timing: [int]):
    statistics = dict()
    
    for time in timing:
        statistics[time] = get_statistics(model, time)
    
    return statistics

def get_statistics(model: Model, days: int) -> int:
    result = 0
    
    for day in range(0, days):
        buses = model.run(flights) #Запуск метода курсирования авто конкретн модели
        
        result += sum(list(map(lambda x: x.flights, buses)))
        
        model.reset_buses()
        
    return result / days

In [5]:
import numpy as np
import matplotlib.pyplot as plt

def show_bar(keys, values, precision: int = 3):
    keys = list(map(lambda x: str(x), keys))
    values = list(map(lambda x: round(x, precision), values))
    
    bars = plt.bar(keys, values)
    
    autolabel(bars)
    
    plt.show()
    
def autolabel(bars, xpos='center'):
    xpos = xpos.lower()
    
    ha = {'center': 'center', 'right': 'left', 'left': 'right'}
    offset = {'center': 0.5, 'right': 0.57, 'left': 0.43}

    for bar in bars:
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()*offset[xpos], 1.01*height,
                '{}'.format(height), ha=ha[xpos], va='bottom')

# Модель без ремонта мелких поломок

In [6]:
from packages.bas_models.src.nonstop_model import NonstopModel

model = NonstopModel(buses)
#Моделирование с заданным количеством дней
statistics = arrange_statistics(model, [1,2,3,4,5,10,20,90])
print(list(statistics.keys()), list(statistics.values()))
#show_bar(list(statistics.keys()), list(statistics.values()))

[1, 2, 3, 4, 5, 10, 20, 90] [2.0, 5.0, 1.6666666666666667, 2.25, 2.8, 2.8, 3.3, 3.3444444444444446]


In [7]:
from examples.bus_models.statistics_with_probs import StatisticsWithProbs
from packages.bas_models.src.nonstop_model import NonstopModel

stats_with_probs = StatisticsWithProbs(NonstopModel([]), 90, 10)

statistics = stats_with_probs.arrange_statistics(probs)
print(list(statistics.keys()), list(statistics.values()))

#show_bar(list(statistics.keys()), list(statistics.values()), 2)

['0.0/0.05', '0.1/0.05', '0.2/0.11', '0.3/0.11', '0.4/0.15', '0.5/0.15', '0.6/0.19', '0.7/0.19', '0.8/0.23', '0.9/0.23', '1.0/0.7'] [10.0, 9.555555555555555, 8.333333333333334, 7.655555555555556, 6.355555555555555, 6.022222222222222, 5.155555555555556, 4.444444444444445, 3.688888888888889, 3.966666666666667, 1.3888888888888888]


# Модель с ремонтом мелких поломок

In [8]:
from packages.bas_models.src.repair_model import RepairModel

model = RepairModel(buses)

statistics = arrange_statistics(model, [1,2,3,4,5,10,20,90])
print(list(statistics.keys()), list(statistics.values()))

#show_bar(list(statistics.keys()), list(statistics.values()))

[1, 2, 3, 4, 5, 10, 20, 90] [6.0, 6.0, 5.666666666666667, 5.5, 6.6, 6.4, 5.85, 6.133333333333334]


In [9]:
from examples.bus_models.statistics_with_probs import StatisticsWithProbs
from packages.bas_models.src.repair_model import RepairModel

stats_with_probs = StatisticsWithProbs(RepairModel([]), 90, 10)

statistics = stats_with_probs.arrange_statistics(probs)
print(list(statistics.keys()), list(statistics.values()))

#show_bar(list(statistics.keys()), list(statistics.values()), 2)

['0.0/0.05', '0.1/0.05', '0.2/0.11', '0.3/0.11', '0.4/0.15', '0.5/0.15', '0.6/0.19', '0.7/0.19', '0.8/0.23', '0.9/0.23', '1.0/0.7'] [10.0, 9.044444444444444, 7.933333333333334, 7.1, 6.022222222222222, 5.011111111111111, 4.188888888888889, 3.2888888888888888, 2.2, 0.9777777777777777, 0.0]


# Вывод

В данных условиях модель без ремонта мелких поломок лучше подходит чем модель с ремонтом мелких поломок.